In [4]:
import torch
import torch.nn as nn
import re
import numpy as np

# ==========================================
# THE MODEL ARCHITECTURE
# ==========================================
class TextCNN(nn.Module):
    def __init__(self, embedding_matrix):
        super().__init__()
        self.relu = nn.ReLU()

        # Init embedding layer
        # We initialize with the correct shape, but the weights will be
        # overwritten by the loaded state_dict immediately after.
        num_embeddings, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)

        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = False # freeze for safety during inference

        # Convs
        self.gram2conv = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2)
        self.pool2 = nn.AdaptiveMaxPool1d(1)
        self.gram3conv = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=3)
        self.pool3 = nn.AdaptiveMaxPool1d(1)
        self.gram4conv = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=4)
        self.pool4 = nn.AdaptiveMaxPool1d(1)
        self.gram5conv = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=5)
        self.pool5 = nn.AdaptiveMaxPool1d(1)

        self.dropout = nn.Dropout(p=0.6)
        self.linear = nn.Linear(in_features=400, out_features=64)
        self.classifier = nn.Linear(in_features=64, out_features=2)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1) # (Batch, Dim, Seq)

        x2 = torch.squeeze(self.pool2(self.relu(self.gram2conv(x))), dim=2)
        x3 = torch.squeeze(self.pool3(self.relu(self.gram3conv(x))), dim=2)
        x4 = torch.squeeze(self.pool4(self.relu(self.gram4conv(x))), dim=2)
        x5 = torch.squeeze(self.pool5(self.relu(self.gram5conv(x))), dim=2)

        total_features = torch.hstack([x2, x3, x4, x5])
        total_features = self.dropout(total_features)
        total_features = self.relu(self.linear(total_features))
        total_features = self.classifier(total_features)
        return total_features



In [18]:
# ==========================================
# THE PREDICTOR
# ==========================================
class SentimentModel:
    def __init__(self, model_path="TextCNN_portable_model.pth"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Loading model on: {self.device}")

        # load the model
        checkpoint = torch.load(model_path, map_location=self.device)

        self.vocab = checkpoint['vocab']
        self.config = checkpoint['config']
        embed_shape = checkpoint['embed_shape']

        dummy_matrix = np.zeros(embed_shape, dtype=np.float32)
        self.model = TextCNN(dummy_matrix)

        state_dict = {k.replace("_orig_mod.", ""): v for k, v in checkpoint['model_state_dict'].items()}
        self.model.load_state_dict(state_dict)

        self.model.to(self.device)
        self.model.eval()
        print("Model loaded successfully.")

    def preprocess(self, text):
        text = re.sub(r"[^a-zA-Z']", " ", text)
        words = text.lower().split()

        # Convert to IDs
        ids = [self.vocab.get(w, 1) for w in words] # 1 is UNK

        # Pad/Truncate
        max_len = self.config['max_len']
        if len(ids) < max_len:
            ids += [0] * (max_len - len(ids)) # 0 is PAD
        else:
            ids = ids[:max_len]

        return torch.tensor([ids], dtype=torch.long).to(self.device)

    def predict(self, text):
        tensor_input = self.preprocess(text)

        with torch.no_grad():
            output = self.model(tensor_input)
            probs = torch.softmax(output, dim=1)
            prediction_idx = torch.argmax(probs, dim=1).item()

        label = self.config['classes'][prediction_idx]
        confidence = probs[0][prediction_idx].item()

        print(f"""
            label: {label},
            confidence: {confidence:.2%},
            probabilities: {probs[0].cpu().numpy()}
            """
        )

How to use:

- Create an object of SentimentModel
- Call the .predict() method

In [19]:
model = SentimentModel()

Loading model on: cpu
Model loaded successfully.


In [20]:
model.predict("This is not a very good car")


            label: Negative,
            confidence: 61.66%,
            probabilities: [0.61658883 0.38341114]
            
